In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import io
import requests
import datetime
import ondemand
from scipy.stats import pearsonr
from pandas.io.common import EmptyDataError
from requests.exceptions import HTTPError
import time
import os
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import argparse
from apiclient.discovery import build
#from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
####Google Analytics Module
class GA(object): 
    def __init__(self, start_date, end_date, symbol):
        self.page_view = None
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
    def initialize_analyticsreporting(self):
        """
          Initializes the analyticsreporting service object.

      Returns:
        analytics an authorized analyticsreporting service object.
        """
        SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
        DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
        CLIENT_SECRETS_PATH = 'client_secrets.json' # Path to client_secrets.json file.
      # Parse command-line arguments.
        parser = argparse.ArgumentParser(
          formatter_class=argparse.RawDescriptionHelpFormatter,
          parents=[tools.argparser])
        flags = parser.parse_args([])

      # Set up a Flow object to be used if we need to authenticate.
        flow = client.flow_from_clientsecrets(
          CLIENT_SECRETS_PATH, scope=SCOPES,
          message=tools.message_if_missing(CLIENT_SECRETS_PATH))

      # Prepare credentials, and authorize HTTP object with them.
      # If the credentials don't exist or are invalid run through the native client
      # flow. The Storage object will ensure that if successful the good
      # credentials will get written back to a file.
        storage = file.Storage('analyticsreporting.dat')
        credentials = storage.get()
        if credentials is None or credentials.invalid:
            credentials = tools.run_flow(flow, storage, flags)
        http = credentials.authorize(http=httplib2.Http())

      # Build the service object.
        analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)
        return analytics

    def get_report(self, analytics, symbol, col_name):
        VIEW_ID = '108608708'#barchart.com view_id
        #set filter pagePath according to different symbol.
        #total_filter = 'ga:pagePath=~(?i)/stocks/quotes/{}/*?'.format(symbol)
        #overview_filter = 'ga:pagePath=~(?i)/stocks/quotes/{}/overview/*?'.format(symbol)
        #build filter exrepression
        base = 'ga:pagePath=~(?i)/stocks/quotes/{}/*?'.format(symbol)
        filters = {}
        for sub_page in col_name:
            if sub_page == 'total_page_views':
                filters[sub_page] = 'ga:pagePath=~(?i)/stocks/quotes/{}.*$;ga:pagePath!~(?i)/story.*$;ga:pagePath!~(?i)/stocks/quotes/{}[A-Za-z0-9\.-].*$'.format(symbol, symbol)
            elif sub_page == 'overview':
                filters[sub_page] = 'ga:pagePath=~(?i)/stocks/quotes/{}/{}.*$,ga:pagePath=~(?i)/stocks/quotes/{}/$,ga:pagePath=~(?i)/stocks/quotes/{}$,ga:pagePath=~(?i)/stocks/quotes/{}\?search={}$;ga:pagePath!~(?i)/story.*$'.\
                format(symbol, sub_page, symbol, symbol, symbol, symbol)
            else:
                filters[sub_page] = 'ga:pagePath=~(?i)/stocks/quotes/{}/{}.*$;ga:pagePath!~(?i)/story.*$'.format(symbol, sub_page)
        final_response = {}
        trail = len(col_name)%5
        num = len(col_name) // 5
        for i in range(num):
            # Use the Analytics Service Object to query the Analytics Reporting API V4.
            response = analytics.reports().batchGet(
          body={
            'reportRequests': [
               {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 0]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 1]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 2]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 3]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                },
                {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[i*5 + 4]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]                
                    
                }
               ]
          }
      ).execute()
            if not final_response.get('reports'):
                final_response['reports'] = response['reports']
            else:
                final_response['reports'] += response['reports']
        for i in range(trail):
            trail_response = analytics.reports().batchGet(
                body = { 'reportRequests': [
                    {
              'viewId': VIEW_ID,
              'dateRanges': [{'startDate': self.start_date, 'endDate': self.end_date}],
              'metrics': [{'expression': 'ga:pageviews'}],
              'filtersExpression' : filters[col_name[num*5 + i]],
              'dimensions':
                     [
                        {
                         'name': 'ga:date'   
                        }                         
                     ]
                   }
                ]
                }).execute()
            if not final_response.get('reports'):
                final_response['reports'] = trail_response['reports']
            else:
                final_response['reports'] += trail_response['reports']
        return final_response
    def json_cut(self, dic, col_name):
        temp_result = {}
        report = dic['reports']
        try:
            #get our date and pageview results
            for item in report[0].get('data', {}).get('rows', []):
                date = item['dimensions'][0][:4] +'-'+ item['dimensions'][0][4:6]+\
                                          '-'+item['dimensions'][0][6:]
                temp_result[date] = item['metrics'][0]['values']
            df = pd.DataFrame.from_dict(temp_result, orient='index')
            df = df.reset_index()
            df = df.rename(index=str, columns={'index': 'date', 0: col_name[0]})
            for i in range(1, len(report)):
                temp_result = {}
                for each in report[i].get('data', {}).get('rows', []):
                    date = each['dimensions'][0][:4] +'-'+ each['dimensions'][0][4:6]+\
                                          '-'+ each['dimensions'][0][6:]
                    temp_result[date] = each['metrics'][0]['values']
                temp_df = pd.DataFrame.from_dict(temp_result, orient = 'index')
                temp_df = temp_df.reset_index()
                temp_df = temp_df.rename(index = str, columns = {'index' : 'date', 0 : col_name[i]})
                df = pd.merge(df, temp_df, how = 'outer', on = 'date')
            df.insert(loc = 1, column = 'symbol', value = self.symbol)
            self.page_view = df
        except:
            return True
    def main(self, symbol):
        analytics = self.initialize_analyticsreporting()
        col_name = ['total_page_views', 'overview', 'performance', 'interactive-Chart', 'technical-chart', \
                   'technical-analysis', 'cheat-sheet', 'opinion' , 'trading-strategies', 'price-history', 'options', \
                    'volatility-greeks', 'covered-calls', 'naked-puts', 'option-spreads', 'news',\
                   'profile', 'sec-filings', 'competitors', 'comparison', 'earnings-estimates', \
                    'analyst-ratings', 'income-statement', 'cash-flow', 'balance-sheet']
        response = self.get_report(analytics, symbol, col_name)
        flag = self.json_cut(response, col_name)
        if flag:
            print(symbol + ' requests pageview failure!!!') 
            

In [54]:
all_symbols = list(pd.read_csv('Highest_change_total.csv')['Symbol'])

In [56]:
#2019-06-03
all_symbols.index('FITB')
all_symbols = all_symbols[3935:]

In [57]:
# pull pageview data here
to_drop_symbol = []
for i in range(0, len(all_symbols)//200 + 1):  # In order to aviod Google Analytics quota or limitation. I need to run by group.
    if i > 0:
        time.sleep(90) # Aviod quota limitation
    temp_symbols = all_symbols[i*200 : (i+1)*200]
    flag = i
    print(datetime.datetime.now())
    for i, item in enumerate(temp_symbols):  
        #print('over here!!!') #test
        if i % 2 == 0:
            time.sleep(1)
        if i % 10 == 0:
            print(i)
        ga = GA('2016-08-01', '2016-12-31', item)
        ga.main(item)
        if ga.page_view is None:
            print(item + ' has no pageview data!!!')
            to_drop_symbol.append(item)
            continue
        if ga.page_view.empty:
            print(item + ' has no pageview data!!!')
            to_drop_symbol.append(item) #I need to drop the symbols if they have no pageview data.
            continue
        ga.page_view = ga.page_view.fillna(0)
        ga.page_view.to_csv('pageview_daily_2016/pageview_daily_{}.csv'.format(item.lower()), index = False)
        #pageview_daily[item] = ga.page_view
    print(datetime.datetime.now())

    

2019-06-05 10:27:52.770165
0
FIXD has no pageview data!!!
FIXX has no pageview data!!!
FKU has no pageview data!!!
10
FLMN has no pageview data!!!
FLN has no pageview data!!!
FLNT has no pageview data!!!
20
FMB has no pageview data!!!
FMCIW has no pageview data!!!
FMHI has no pageview data!!!
30
FNK has no pageview data!!!
FNKO has no pageview data!!!
FNX has no pageview data!!!
FNY has no pageview data!!!
FOCS has no pageview data!!!
40
50
FPAYW has no pageview data!!!
60
FRSX has no pageview data!!!
FSCT has no pageview data!!!
70
FSZ has no pageview data!!!
FTA has no pageview data!!!
FTC has no pageview data!!!
FTCS has no pageview data!!!
FTDR has no pageview data!!!
FTFT has no pageview data!!!
80
FTGC has no pageview data!!!
FTRI has no pageview data!!!
FTSL has no pageview data!!!
FTSM has no pageview data!!!
FTSV has no pageview data!!!
FTXG has no pageview data!!!
90
FTXH has no pageview data!!!
FTXL has no pageview data!!!
FTXN has no pageview data!!!
FTXO has no pageview da

HttpError: <HttpError 429 when requesting https://analyticsreporting.googleapis.com/v4/reports:batchGet?alt=json returned "Quota Error: profileId 108608708 has exceeded the daily request limit.">

In [5]:
# Using this function to fill missed date with zero pageviews
def temp_parallel(item):
    try:
        df = pd.read_csv('pageview_daily_2017/pageview_daily_{}.csv'.format(item.lower()))
    except FileNotFoundError:
        return
    df.fillna(0, inplace = True)
    idx = pd.date_range('2017-07-01', '2017-12-31')
    df.index = pd.DatetimeIndex(df['date'])
    df = df.reindex(idx, fill_value = 0)
    df.rename(index = str, columns = {'date' : 'to_drop'}, inplace = True)
    df.drop(['to_drop'], axis = 1, inplace = True)
    df.reset_index(inplace = True)
    df['index'] = pd.to_datetime(df['index']).dt.date
    df.rename(index = str, columns = {'index' : 'date'}, inplace = True)
    df['symbol'] = item
    df.to_csv('pageview_daily_2017/pageview_daily_{}.csv'.format(item.lower()), index = False)  

In [6]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
pool = ThreadPool(10)
#pool.map(temp_parallel, symbols)  #important, no run except permission
pool.close()
pool.join()

In [8]:
#combine data in 2017 with 2018
for item in symbols:
    try:
        df1 = pd.read_csv('pageview_daily_2017/pageview_daily_{}.csv'.format(item.lower()))
        df2 = pd.read_csv('pageview_daily_2018/pageview_daily_{}.csv'.format(item.lower()))
    except FileNotFoundError:
        continue
    df = pd.concat([df1, df2], sort = False)
    df = df.to_csv('pageview_daily/pageview_daily_{}.csv'.format(item.lower()), index = False)

In [15]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
class Get_Summary(object):
    def __init__(self, start_date, end_date):
        #we need to catch error, if dates are illeagle
        self.start_date = start_date
        self.end_date = end_date
        self.page_views = dict()
        self.summary_res = pd.DataFrame([], columns = ['Symbol', 'startDate', 'endDate', 'Total Page View', \
                                          'Average Daily Page View', 'Std Daily Page View', 'Lowest Daily Page View', \
                                         '25% Daily Page View', '50% Daily Page View', '75% Daily Page View', \
                                         'Highest Daily Page View', 'Market_Cap(billion)'])
    def get_mkt_cap(self, symbol):

        url_shares = 'https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=OnDemand&symbols={}&fields=lastQtrEPS%2CannualEPS%2CttmEPS'.\
                format(symbol)
        try:
            fundmantal = pd.read_csv(url_shares)
            if fundmantal.empty:
                print(symbol + ' does not have public market value!!!')
                return None
            else:
                shares = fundmantal.loc[0, 'sharesOutstanding']/1000000 # Change to billion
        except HTTPError as he:
            print('getFinancialHighlights API Error!!! Run again')
        url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'.\
                format(symbol, self.end_date, self.end_date)
        try:
            price = pd.read_csv(url_price)
            if price.empty:
                print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(end_date))
                mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
                return float(mkt['marketCapitalization'])/1000000
        except:
            print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(self.end_date))
            mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
            return float(mkt['marketCapitalization'])/1000000            
        price_end = price['close']
        return float(price_end)*shares
    def combine_page(self):
        midcap = pd.read_csv('mid_cap_symbols.csv')
        all_symbols = list(midcap['Symbol'])
        for item in all_symbols:
            try:
                temp = self.page_views[item]
            except:
                continue
            self.summary_res = pd.concat([self.summary_res, temp], sort = False)
    def get_data(self, item):
        try:
            pageview_daily = pd.read_csv('pageview_daily/pageview_daiyl_{}.csv'.format(item.lower()))
        except FileNotFound:
            return
        total = pageview_daily['total_page_views'].apply(int)
        total_pageview = sum(total)
        temp_res = pd.DataFrame(total.describe()).T
        temp_res = temp_res.reset_index()
        temp_res['startDate'] = ga.start_date
        temp_res['endDate'] = ga.end_date
        temp_res['total_page'] = total_pageview
        temp_res['symbol'] = ga.symbol
        temp_res['mkt(billion)'] = self.get_mkt_cap(item)
        temp_res = temp_res.drop(['index', 'count'], axis = 1)
        temp_res = temp_res[['symbol', 'startDate', 'endDate', 'total_page', 'mean', 'std', 'min', '25%', '50%', '75%',\
                            'max', 'mkt(billion)']]
        temp_res['mean'] = temp_res['mean'].round(2)
        temp_res['std'] = temp_res['std'].round(2)
        if temp_res['mkt(billion)'].isnull:
            pass
        else:
            temp_res['mkt(billion)'] = temp_res['mkt(billion)'].round(2)
        temp_res = temp_res.rename(index = str, columns = {'symbol' : 'Symbol', 'total_page' : 'Total Page View',\
                                                          'mean' : 'Average Daily Page View', \
                                                          'std' : 'Std Daily Page View', 'min' : 'Lowest Daily Page View',\
                                                          '25%' : '25% Daily Page View', '50%' : '50% Daily Page View', \
                                                          '75%' :  '75% Daily Page View', 'max' : 'Highest Daily Page View',\
                                                          'mkt(billion)' : 'Market_Cap(billion)'})
        self.page_views[item] = temp_res
    def main(self, symbols):
        """
        to_drop_symbol = self.get_page_view(symbols)
        #drop here
        for item in to_drop_symbol:
            symbols.remove(item)
        self.combine_page(symbols)        
        """
        print(datetime.datetime.now())
        pool = ThreadPool(30)
        pool.map(self.get_data, all_symbols) #important, no run except permission
        pool.close()
        pool.join()
        self.combine_page()
        print(datetime.datetime.now())

In [10]:
#combine pageview and price.
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
pageview_price = Manager().dict()
def page_price_parallel(item):
    temp_pageview_price = pd.DataFrame([])
    try:
        pageview_daily = pd.read_csv('pageview_daily/pageview_daily_{}.csv'.format(item.lower()))
    except FileNotFoundError:
        return 
    temp_pageview_price['Date'] = pageview_daily['date']
    temp_pageview_price['Symbol'] = item
    temp_pageview_price['PageView'] = pageview_daily['total_page_views'].apply(int)
    url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'.\
                format(item, '2017-07-01', '2018-12-31')
    try:
        whole_price = pd.read_csv(url_price)
    except EmptyDataError:
        print('{} has no price data during this time!!!'.format(item))
        whole_price = pd.DataFrame([], columns = ['tradingDay', 'close'])
    whole_price.rename(index = str, columns = {'tradingDay' : 'Date', 'close' : 'Close Price'}, inplace = True)
    temp_pageview_price = temp_pageview_price.merge(whole_price, how = 'outer', on = 'Date')
    price_col_name = list(whole_price.columns.values)
    name_to_drop = [n for n in price_col_name if n not in ['Date', 'Close Price']]
    temp_pageview_price.drop(name_to_drop, axis = 1, inplace = True)
    #drop observations for no trading.
    temp_pageview_price = temp_pageview_price[pd.notnull(temp_pageview_price['Close Price'])]
    temp = temp_pageview_price['PageView'] 
    temp_std = temp.rolling(100).std()
    temp_pageview_price['PageView_Change_in_Std_1'] = \
        ((temp.rolling(1).mean() - temp.rolling(100).mean().shift(1))/temp_std.shift(1)).round(2)
    temp_pageview_price['PageView_Change_in_Std_3'] = \
        ((temp.rolling(3).mean() - temp.rolling(100).mean().shift(3))/temp_std.shift(3)).round(2)
    pageview_price[item] = temp_pageview_price
    temp_pageview_price.to_csv('pageview_price/pageview_price_{}.csv'.format(item.lower()), index = False)


In [11]:
pool = ThreadPool(20)
pool.map(page_price_parallel, symbols)
pool.close()
pool.join()

EMITF has no price data during this time!!!
HMNY has no price data during this time!!!
FOX has no price data during this time!!!
FOXA has no price data during this time!!!
RGSE has no price data during this time!!!
OHGI has no price data during this time!!!
LINK has no price data during this time!!!
THST has no price data during this time!!!


In [25]:
all_symbols = all_symbols[:]

In [27]:
len(all_symbols)

6110

In [28]:
for each in all_symbols:
    #fix price change forward problem
    try:
        df = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(each.lower()))
    except:
        continue
    df.drop(['PageView_level_in_Std'], axis = 1, inplace = True) 
    temp = df['PageView']
    temp_res = ((temp - temp.rolling(100).mean())/temp.rolling(100).std()).round(2)
    df.insert(loc = 4, column = 'PageView_level_in_Std(rolling_100days)', value = temp_res)
    temp2 = df['PageView_level_in_Std(rolling_100days)']
    df.insert(loc = 5, column = 'PageView_3Days_in_Std', value = temp2.rolling(3).mean().round(2))
    df['PageView_5Days_in_Std'] = temp2.rolling(5).mean().round(2)
    df['PageView_30Days_in_Std'] = temp2.rolling(30).mean().round(2)
    df.insert(loc = 10, column = 'Close Price Change_Forward_3day', value = df['Close Price'].diff(3))
    df.insert(loc = 11, column = 'Close Price Percent Change_Forward_3day(%)', \
          value = (df['Close Price'].pct_change(3)*100).round(2))
    df['Close Price Change_Forward_3day'] = df['Close Price Change_Forward_3day'].shift(-3)
    df['Close Price Percent Change_Forward_3day(%)'] = df['Close Price Percent Change_Forward_3day(%)'].shift(-3)   
    df.to_csv('pageview_price/pageview_price_{}.csv'.format(each.lower()), index = False)

In [18]:
from multiprocessing import Pool as ThreadPool
from multiprocessing import Manager
class Generate_Leaders():
    def __init__(self): 
        self.pageview_change = Manager().dict()
        self.my_date = None
    def get_mkt_cap(self, symbol, start_date, end_date):

        url_shares = 'https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=OnDemand&symbols={}&fields=lastQtrEPS%2CannualEPS%2CttmEPS'.\
                format(symbol)
        try:
            fundmantal = pd.read_csv(url_shares)
            if fundmantal.empty:
                print(symbol + ' does not have public market value!!!')
                return None
            else:
                shares = fundmantal.loc[0, 'sharesOutstanding']/1000000 # Change to billion
        except HTTPError as he:
            print('getFinancialHighlights API Error!!! Run again')
        url_price = 'https://ondemand.websol.barchart.com/getHistory.csv?apikey=OnDemand&symbol={}&type=daily&startDate={}&endDate={}&maxRecords=1000&interval=60&order=asc&sessionFilter=EFK&splits=true&dividends=true&volume=sum&nearby=1&jerq=true&exchange=NYSE%2CAMEX%2CNASDAQ&backAdjust=false&daysToExpiration=1&contractRoll=expiration'.\
                format(symbol, end_date, end_date)
        try:
            price = pd.read_csv(url_price)
            if price.empty:
                print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(end_date, start_date))
                mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
                return float(mkt['marketCapitalization'])/1000000
        except:
            print(symbol + ' has no trading price on {}, the mkt_cap is obtained by getfinancialhighlight API'.format(end_date, start_date))
            mkt = pd.read_csv('https://ondemand.websol.barchart.com/getFinancialHighlights.csv?apikey=ondemand&symbols={}'.format(symbol))
            return float(mkt['marketCapitalization'])/1000000            
        price_end = price['close']
        return float(price_end)*shares
    def highest_pageview(self, start_date, end_date):
        all_symbols = list(pd.read_csv('Highest_change_total.csv')['Symbol'])
        to_combine = pd.DataFrame([], columns = ['Symbol', 'startDate', 'endDate', 'Total Page View', 'Market_Cap(billion)'])
        for i, item in enumerate(all_symbols):
            try:
                pageview_daily = pd.read_csv('pageview_daily/pageview_daily_{}.csv'.format(item.lower()))
            except FileNotFoundError:
                print('{} has no pageview data!!!'.format(item))
                continue
            pageview_daily['datetime'] =  pd.to_datetime(pageview_daily['date'])
            pageview_daily = pageview_daily.set_index(['datetime'])
            start_datetime = pd.to_datetime(start_date)
            end_datetime = pd.to_datetime(end_date)
            #print(start_date)
            pageview_daily = pageview_daily.loc[start_datetime : end_datetime]
            #print(pageview_daily)
            total = pageview_daily['total_page_views'].apply(int)
            total_pageview = sum(total)
            #temp_res = pd.DataFrame(total.describe()).T
            temp_res = pd.DataFrame([], columns = ['Symbol', 'startDate', 'endDate', 'Total Page View', 'Market_Cap(billion)'])
            #temp_res = temp_res.reset_index()
            temp_res.loc[0, 'Symbol'] = item
            temp_res['startDate'] = start_date
            temp_res['endDate'] = end_date
            temp_res['Total Page View'] = total_pageview
            temp_res['Market_Cap(billion)'] = self.get_mkt_cap(item, start_date, end_date)
            if temp_res['Market_Cap(billion)'].isnull().values.any():
                pass
            else:
                temp_res['Market_Cap(billion)'] = temp_res['Market_Cap(billion)'].round(2)
            to_combine = pd.concat([to_combine, temp_res])
        #to_combine['Market_Cap(billion)'] = to_combine['Market_Cap(billion)'].round(2)
        to_combine.reset_index(inplace = True)
        to_combine.drop(['index'], axis = 1, inplace = True)
        to_combine.sort_values(by = ['Total Page View'], ascending = False, inplace = True)
        return to_combine
    def parallel(self, item):
        try:
            pageview_price = pd.read_csv('pageview_price/pageview_price_{}.csv'.format(item.lower()))
        except FileNotFoundError:
            print('{} has no pageview data!!!'.format(item))
            return
        if pageview_price.empty:
            return
        #print(datetime.datetime.now())
        temp_res = pageview_price.set_index(['Date'])
        try:
            temp_res = temp_res.loc[[self.my_date]]
            temp_res.reset_index(col_fill = 'Date', inplace = True)
        except:
            print('{} has no trading on {}!!!'.format(item, self.my_date))
            return
        temp_res.loc[0, 'Market_Cap(billion)'] = self.get_mkt_cap(item, self.my_date, self.my_date)
        if temp_res['Market_Cap(billion)'].isnull().values.any():
            pass
        else:
            temp_res['Market_Cap(billion)'] = temp_res['Market_Cap(billion)'].round(2)
        self.pageview_change[item] = temp_res 
    def highest_change(self, my_date):
        all_symbols = list(pd.read_csv('Highest_change_total.csv')['Symbol'])
        self.my_date = my_date
        print(datetime.datetime.now())
        pool = ThreadPool(50)
        pool.map(self.parallel, all_symbols)
        pool.close()
        pool.join()
        print(datetime.datetime.now())
        """

                for i, item in enumerate(all_symbols):
            if i % 500 == 0:
                print(i)
            self.parallel(item)
        """
